In [72]:
import os

if "src" not in os.listdir():
    os.chdir("../../../")

import numpy as np
from dateutil import parser
import json
import pickle
from keras.models import load_model
from datetime import timezone
import pandas as pd

from src.lib.stock_dataset import read_stock_dataset
from src.lib.activations import negative_softmax

In [73]:
path = "results/1_lstm_ptrade/"
keys = "alpaca_config.json"

model = load_model(
    os.path.join(path, "model.h5"),
    custom_objects={
        "negative_softmax": negative_softmax
    }
)

with open(os.path.join(path, "config.json"), "r") as f:
    config = json.load(f)

n_time_steps = config["n_time_steps"]
symbols = config["symbols"]
target_column = config["target_column"]

with open(os.path.join(path, "scaler.pkl"), "rb") as f:
    scaler = pickle.load(f)

ta_preprocess_candles = 296

In [74]:
dataset = read_stock_dataset(symbols, target_column, n_time_steps, transform=True)

In [75]:
dataset = dataset.apply_standard_scaler(scaler)

In [78]:
path = "results/1_lstm_ptrade/Xs"
files = os.listdir(path)
for file_name in files[::-1]:
    file_path = os.path.join(path, file_name)
    X = np.load(file_path)
    # X[0] = scaler.transform(X[0])
    time = parser.parse(file_name[:-len(".npy")])
    time = time.replace(second=0, microsecond=0)
    i = dataset.X.index.get_loc(time)
    df_chunk = dataset.X.iloc[i-n_time_steps: i]
    X_hist = df_chunk.to_numpy()[np.newaxis, :, :]

    # assert (X == X_hist).all()
    print(X.shape, X_hist.shape)
    m = 6
    diff = np.abs(X_hist - X)
    diff = diff[0, :, :].mean(0)
    for symbol, value in zip(dataset.X.columns, diff):
        print(symbol, value)
    # print((X_hist[0, :, m] - X[0, :, m]).sum())
    break

(1, 64, 26) (1, 64, 26)
SPY_volume 216473.35136359156
SPY_trades 1781.6271198366207
SPY_oc_ret 0.7304869431474138
SPY_pdiff_open 0.7306368117909421
SPY_pdiff_high 0.4521792960762197
SPY_pdiff_low 0.33099236074348476
SPY_pdiff_vwap 0.7523634530763113
SPY_ma_50 0.4742175499668744
SPY_ma_200 1.2380929776932839
SPY_rsi 45.427305704758695
SPY_macd 0.8545756295828085
SPY_bbands_b 0.18871185246020178
SPY_bbands_p 0.8176656018632554
SPXS_volume 22762.110568428106
SPXS_trades 56.75940344576772
SPXS_oc_ret 0.5948513414613346
SPXS_pdiff_open 0.5944947043309758
SPXS_pdiff_high 0.4451214486937081
SPXS_pdiff_low 0.4373596207496522
SPXS_pdiff_vwap 0.5877960717680195
SPXS_ma_50 0.31664502701791636
SPXS_ma_200 0.7940813814425918
SPXS_rsi 55.03419740945672
SPXS_macd 0.277991477300929
SPXS_bbands_b 0.3359206078857252
SPXS_bbands_p 0.8753314897089739
